In [1]:
import os
import re 
import glob
import openpyxl

In [2]:
# C:\Users\kazit\Downloads\parent  

In [35]:
import os
import openpyxl

# Function to create an Excel file with subfolder names recursively
def create_excel_file(parent_folder, worksheet, start_row=2):
    for root, dirs, files in os.walk(parent_folder):
        for dir_name in dirs:
            worksheet.cell(row=start_row, column=1, value=dir_name)
            start_row += 1

    # Save the Excel file
    excel_file = os.path.join(parent_folder, "subfolders.xlsx")
    worksheet.parent.save(excel_file)

    return excel_file

# Input the parent folder path
parent_folder = input("Enter the path of the parent folder: ")

# Create the Excel file with subfolder names recursively
workbook = openpyxl.Workbook()
worksheet = workbook.active
worksheet.title = "Subfolders"

# Add headers
worksheet.cell(row=1, column=1, value="Subfolder Name")
worksheet.cell(row=1, column=2, value="MAIN")
worksheet.cell(row=1, column=3, value="ASIN")

excel_file = create_excel_file(parent_folder, worksheet)

print(f"Excel file 'subfolders.xlsx' created in '{parent_folder}'")
print("Please fill in the ASIN and MAIN columns as needed.")
print("Once you've filled in the Excel file, run the utility again to rename the files.")


Enter the path of the parent folder: C:\Users\kazit\Downloads\Compressed\Animals - Humour
Excel file 'subfolders.xlsx' created in 'C:\Users\kazit\Downloads\Compressed\Animals - Humour'
Please fill in the ASIN and MAIN columns as needed.
Once you've filled in the Excel file, run the utility again to rename the files.


In [38]:
import os
import openpyxl

# Function to read the Excel file and return subfolder data
def read_excel_file(excel_file):
    subfolder_data = []

    if os.path.exists(excel_file):
        workbook = openpyxl.load_workbook(excel_file)
        worksheet = workbook.active

        for row in worksheet.iter_rows(min_row=2, values_only=True):
            subfolder_data.append(row)

    return subfolder_data

# Function to rename files within a subfolder based on subfolder data
def rename_files(subfolder_path, main_file_name, asin_number):
    for root, _, files in os.walk(subfolder_path):
        for filename in files:
            main_exists = False

            for filename in os.listdir(subfolder_path):
                file_path = os.path.join(subfolder_path, filename)

                # Skip directories
                if os.path.isdir(file_path):
                    rename_files(file_path, main_file_name, asin_number)  # Recursively rename files in subfolders
                    continue

                if main_file_name and filename.upper() == main_file_name.upper():
                    main_exists = True
                    continue  # Skip renaming the main file if it exists

                base, ext = os.path.splitext(filename)
                if main_file_name and (main_file_name.upper().endswith("MAIN_1") or ext.upper().endswith("MAIN_1")):
                    new_filename = f"{asin_number}.MAIN{ext}"
                # Check if "_1" is at the end of the file name or extension
                elif main_file_name and (main_file_name.upper().endswith("_1") or ext.upper().endswith("_1")):
                    new_filename = f"{asin_number}.MAIN{ext}"
                else:
                    if main_exists:
                        new_filename = f"{asin_number}.PT{extract_numerical_value(base):02d}{ext}"
                    else:
                        new_filename = f"{asin_number}.MAIN{ext}"
        
                src_file_path = os.path.join(subfolder_path, filename)
                dst_file_path = os.path.join(subfolder_path, new_filename)

                if not os.path.exists(dst_file_path):
                    os.rename(src_file_path, dst_file_path)
                    print(f"Renamed: {filename} -> {new_filename}")
                else:
                    break
                    


# Function to rename files recursively in all subfolders
def rename_files_recursively(parent_folder, subfolder_data):
    for subfolder_name, main_file_name, asin_number in subfolder_data:
        found = False
        for root, dirs, _ in os.walk(parent_folder):
            if subfolder_name.lower() in [d.lower() for d in dirs]:
                found = True
                subfolder_path = os.path.join(root, subfolder_name)
                rename_files(subfolder_path, main_file_name, asin_number)
                break
        if not found:
            print(f"Subfolder '{subfolder_name}' not found in '{parent_folder}'.")

# Input the parent folder path
parent_folder = input("Enter the path of the parent folder: ")

excel_file = os.path.join(parent_folder, "subfolders.xlsx")

subfolder_data = read_excel_file(excel_file)

if subfolder_data:
    rename_files_recursively(parent_folder, subfolder_data)
else:
    print(f"No subfolder data found in '{excel_file}'.")


Enter the path of the parent folder: C:\Users\kazit\Downloads\Compressed\Animals - Humour
Renamed: GSC24498_1.jpg -> 1.MAIN.jpg
Renamed: GSC24538_1.jpg -> 2.MAIN.jpg
Renamed: GSC24540_1.jpg -> 3.MAIN.jpg
Renamed: GSC24541_1.jpg -> 4.MAIN.jpg
Renamed: GSC24204_1.jpg -> 5.MAIN.jpg
Renamed: GSC24264_1.jpg -> 6.MAIN.jpg
Renamed: GSC24272_1.jpg -> 7.MAIN.jpg


In [39]:
import os
import openpyxl

# Function to read the Excel file and return subfolder data
def read_excel_file(excel_file):
    subfolder_data = []

    if os.path.exists(excel_file):
        workbook = openpyxl.load_workbook(excel_file)
        worksheet = workbook.active

        for row in worksheet.iter_rows(min_row=2, values_only=True):
            subfolder_data.append(row)

    return subfolder_data

# Function to rename files within a subfolder based on subfolder data
def rename_files(subfolder_path, main_file_name, asin_number):
    main_exists = False
    files_to_rename = []

    for filename in os.listdir(subfolder_path):
        file_path = os.path.join(subfolder_path, filename)

        # Skip directories
        if os.path.isdir(file_path):
            rename_files(file_path, main_file_name, asin_number)  # Recursively rename files in subfolders
            continue

        if main_file_name and filename.upper() == main_file_name.upper():
            main_exists = True
            continue  # Skip renaming the main file if it exists

        if "main" not in filename.lower():
            files_to_rename.append(filename)

    # Sort the list of files and rename them in the order of "ASIN_Number.PT01, PT02, PT03, ..."
    pt_number = 1
    for filename in sorted(files_to_rename):
        base, ext = os.path.splitext(filename)
        new_filename = f"{asin_number}.PT{pt_number:02d}{ext}"

        src_file_path = os.path.join(subfolder_path, filename)
        dst_file_path = os.path.join(subfolder_path, new_filename)

        os.rename(src_file_path, dst_file_path)
        print(f"Renamed: {filename} -> {new_filename}")
        print('\n')
        pt_number += 1

# Function to rename files recursively in all subfolders
def rename_files_recursively(parent_folder, subfolder_data):
    for subfolder_name, main_file_name, asin_number in subfolder_data:
        found = False
        for root, dirs, _ in os.walk(parent_folder):
            if subfolder_name.lower() in [d.lower() for d in dirs]:
                found = True
                subfolder_path = os.path.join(root, subfolder_name)
                rename_files(subfolder_path, main_file_name, asin_number)
                break
        if not found:
            print(f"Subfolder '{subfolder_name}' not found in '{parent_folder}'.")

# Input the parent folder path
parent_folder = input("Enter the path of the parent folder: ")

excel_file = os.path.join(parent_folder, "subfolders.xlsx")

subfolder_data = read_excel_file(excel_file)

if subfolder_data:
    rename_files_recursively(parent_folder, subfolder_data)
else:
    print(f"No subfolder data found in '{excel_file}'.")


Enter the path of the parent folder: C:\Users\kazit\Downloads\Compressed\Animals - Humour
Renamed: GSC24498_2.jpg -> 1.PT01.jpg


Renamed: GSC24498_3.jpg -> 1.PT02.jpg


Renamed: GSC24498_4.jpg -> 1.PT03.jpg


Renamed: GSC24498_5.jpg -> 1.PT04.jpg


Renamed: GSC24498_6.jpg -> 1.PT05.jpg


Renamed: GSC24498_7.jpg -> 1.PT06.jpg


Renamed: GSC24498_8.jpg -> 1.PT07.jpg


Renamed: GSC24538_2.jpg -> 2.PT01.jpg


Renamed: GSC24538_3.jpg -> 2.PT02.jpg


Renamed: GSC24538_4.jpg -> 2.PT03.jpg


Renamed: GSC24538_5.jpg -> 2.PT04.jpg


Renamed: GSC24538_6.jpg -> 2.PT05.jpg


Renamed: GSC24538_7.jpg -> 2.PT06.jpg


Renamed: GSC24538_8.jpg -> 2.PT07.jpg


Renamed: GSC24540_2.jpg -> 3.PT01.jpg


Renamed: GSC24540_3.jpg -> 3.PT02.jpg


Renamed: GSC24540_4.jpg -> 3.PT03.jpg


Renamed: GSC24540_5.jpg -> 3.PT04.jpg


Renamed: GSC24540_6.jpg -> 3.PT05.jpg


Renamed: GSC24540_7.jpg -> 3.PT06.jpg


Renamed: GSC24540_8.jpg -> 3.PT07.jpg


Renamed: GSC24541_2.jpg -> 4.PT01.jpg


Renamed: GSC24541_3.jpg -> 4.P